In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
from pyod.models.copod import COPOD 
from pyod.models.iforest import IForest
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
import pyod
import pandas as pd
import numpy as np
import prince
import ipdb


class Data:
    """Data class holding a column by column profile and index flagged as low quality data"""

    def __init__(self, path):
        """
        Args:
            data (CSV, JSON, SQL): data set.
        """
        if check_extension(path) == "none":
            raise TypeError("data should be of provided as .csv or .json or .sql file")

        self.data = _to_DataFrame(path)
        self._profile = None
        self._good_index = list(range(self.data.shape[0]))
        self._bad_index = pd.DataFrame(columns=['idx', 'column', 'errtype'])

    @property
    def profile(self):
        """getter for private attribute _profile

        Raises:
            Exception: If profile is not yet initialize it raise an error.

        Returns:
            Object: Profile object
        """
        if self._profile is None:
            raise Exception('profile is None')
        return self._profile
    
    def set_profile(self):
        """profile setter, to use after initializing the instance.
        """
        profile = {column: Profile(self, column) for column in self.data.columns}
        self._profile = profile

    @property
    def good_index(self):
        """getter for private attribute _good_index

        Returns:
            list: list of good indexes to use for ML training purposes
        """
        return self._good_index

    @property
    def bad_index(self):
        """getter for private attribute for _bad_index

        Returns:
            dataFrame: dataFrame containing error indexes and if applicable column and an explaination of the error
        """
        return self._bad_index
    
    @bad_index.setter
    def bad_index(self, list_idx):
        """setter if private attribute bad_index

        Args:
            list_idx (dataFrame): used as bad_index = bad_index.append(df)
        """
        self._bad_index = list_idx

    @good_index.setter
    def good_index(self, list_idx):
        """setter for private attribute _good_index

        Args:
            list_idx (list): list of good index to replace the previous one

        Raises:
            ValueError: if length is greater than the initial dataFrame raises Valueerror
        """
        if len(list_idx) > self.data.shape[0]:
            raise ValueError('Index length must be smaller than the length of the dataframe')
        self._good_index = list_idx

    def get_str_col(self):
        """return names of string columns of the dataFrame, raises an exception if profile is not set.

        Returns:
            [list]: list of string columns
        """
        col_list = []
        for column in self.data.columns:
            if self.profile[column]._col_type == type(str()):
                col_list.append(column)
        return col_list

    def get_nbr_col(self):
        """return names of number columns of the dataFrame, raises an exception if profile is not set.

        Returns:
            [list]: list of number columns
        """
        col_list = []
        for column in self.data.columns:
            if self.profile[column]._col_type in [type(int()), type(float())]:
                col_list.append(column)
        return col_list
        
    def push_bad_index(self, list_idx): #Find a better method name
        """decrepated, not sure if will be used or not.

        Args:
            list_idx ([type]): [description]
        """
        for elem in list_idx:
            try:
                self.bad_index.append(elem)
            except:
                pass

    def firstpass(self):
        """Push into self.bad_index the indexes and error types of data.
        This first pass detects duplicated data, typo and extreme values
        """
        # Deterministic pass
        n_duped_idx = ~_duplicated_idx(self.data)

        for index in n_duped_idx[~n_duped_idx].index.values.tolist():
            self.bad_index = self.bad_index.append({'idx': index, 'column': 'All', 'errtype': 'duplication'}, ignore_index=True)
        
        # Probabilistic pass
        for column in self.get_str_col(): # Columns of strings only
            if data.profile[column]._uniqueness <= 0.005: # Filter column with too many different words
                clean_df = self.data[n_duped_idx]
                clean_df = clean_df[column][clean_df[column].notna().values]
                idx = index_uncorrect_grammar(clean_df) #get the non duped indexes and not na from a column
                idx = clean_df.iloc[idx].index
                
                for index in idx:
                    row = {'idx': index, 'column': column, 'errtype': 'typo'}
                    self.bad_index = self.bad_index.append(row, ignore_index=True)

        for column in self.get_nbr_col(): # Columns of numbers only
            clean_df = self.data[n_duped_idx]
            clean_df = clean_df[column][clean_df[column].notna().values]
            idx = proba_model(clean_df, self.profile[column]._mean, self.profile[column]._std)
            idx = clean_df[idx].index

            for index in idx:
                row = {'idx': index, 'column': column, 'errtype': 'extreme value'}
                self.bad_index = self.bad_index.append(row, ignore_index=True)

class Profile:
    """A profile for a dataframe column.
    """

    def __init__(self, Data, column):
        self._emptiness = _is_none(Data.data, column)
        self._size = Data.data[column].shape[0]
        self._uniqueness = _is_unique(Data.data, column)
        self._col_type = check_data_type(Data.data[column])
        if self._col_type == type(str()):
            pass
        if self._col_type in [type(int()), type(float())]:
            self._min = Data.data[column].min()
            self._max = Data.data[column].max()
            self._mean = Data.data[column].mean()
            self._std = Data.data[column].std()

    @property
    def emptiness(self):
        """getter of private attribute _emptiness

        Returns:
            [float]: ratio of na inside column
        """
        return self._emptiness

    @emptiness.setter
    def emptiness(self, value):
        """setter of private attribute _emptiness

        Args:
            value (float): ratio of na inside column
        """
        self._emptiness = value
    
    @property
    def size(self):
        """getter of private attribute _size

        Returns:
            [int]: size of the column
        """
        return self._size

    @size.setter
    def size(self, value):
        """setter of private attribute _size

        Args:
            value ([int]): size of the column 
        """ # To note : make truly private
        self._size = value

    @property
    def uniqueness(self):
        """getter of private attribute _uniqueness

        Returns:
            [float]: ratio of unique element inside column
        """
        return self._uniqueness

    @uniqueness.setter
    def uniqueness(self, value):
        self._uniqueness = value

    @property
    def col_type(self):
        """getter of private attribute _col_type

        Returns:
            [type]: returns type python object of the type of the column 
        """
        return self._col_type



import numpy as np
import pandas as pd
import re

from dateutil.parser import parse

from sklearn.cluster import AffinityPropagation
from difflib import SequenceMatcher



"""df = pd.read_csv("logs.csv")  # read data

df = df.set_index("d")  # to re-index with a column 'd'
df = df.sort_index()  # to sort with respect to the index """


def check_extension(data):
    """check if the extension of data is within CSV, JSON or SQL

    Args:
        data (): data set

    Returns:
        type of allowed extension or none.
    """
    if re.search("\.csv$", data, flags=re.IGNORECASE):
        return "csv"
    if re.search("\.json$", data, flags=re.IGNORECASE):
        return "json"
    if re.search("\.sql$", data, flags=re.IGNORECASE):
        return "sql"
    if re.search("\.xlsx$", data, flag=re.IGNORECASE):
        return "xlsx"
    return "none"


def _to_DataFrame(data):
    """read data and transform it to DataFrame

    Args:
        data (csv, json, sql, xlsx): data

    Returns:
        Dataframe
    """

    ext = check_extension(data)
    assert ext != "none"
    f_dict = {
        "csv": pd.read_csv,
        "json": pd.read_json,
        "sql": pd.read_sql,
        "xlsx": pd.read_excel,
    }
    df = f_dict[ext](data)
    return df


def get_metadata(df):
    """read a dataframe and generate relevant metadata such as columns types etc

    Args:
        df (DataFrame): data

    Returns:
        dict: {name_of_column: metadata_associated}
    """
    metadata = []
    for column in df:
        metadata.append(check_data_type(df[column]))
    return metadata


def check_data_type(column):
    """check type in a column which type is it using a voting method from all the non na data

    Args:
        column (pandas.core.series.Series): column from a dataframe

    Returns:
        [type]: [description]
    """
    types_dict = {}
    for e in column[column.notna()]:
        if type(e) not in types_dict:
            types_dict[type(e)] = 1
        else:
            types_dict[type(e)] += 1
    if len(types_dict) != 0:
        return max(types_dict, key=types_dict.get)
    else:
        return


def _is_date(string, fuzzy=False):
    """check if a given string is a date and return the date if true and raise a ValueError if false

    Args:
        string (string): string to check
        fuzzy (bool, optional): Enable a more lenient search in the string. Defaults to False.

    Raises:
        ValueError: raised when string is not likely to be a date

    Returns:
        string: datetime as a string
    """
    try:
        pd.to_datetime(string)
        return pd.to_datetime(string)

    except ValueError:
        raise ValueError


def _is_duplicated(df):
    """Find duplicated row and return dataframe without the duplication

    Args:
        df (pandas.DataFrame): data frame

    Returns:
        duplicated_row: the duplicated rows
        df_clean: the DataFrame without the duplicated rows
    """
    df_new = df.drop(["Unnamed: 0"], axis=1)
    duplicated_row = df[df_new.duplicated()]  # duplicated row
    df_clean = df[~df_new.duplicated()]  # without duplication row
    return df_clean, duplicated_row


def _is_unique(df, col_name=""):
    """verify uniqueness over a specified column, and find the uniqueness coefficient

    Args:
        df (pandas.DataFrame): Data Frame.
        col_name (str, optional): Column name. Defaults to "".

    Returns:
        ratio : 1 - (number of repeated data in a column)/card(the column)
                if 1 means all values are unique
    """
    df_clean, _ = _is_duplicated(df)
    return df_clean[col_name].nunique() / df_clean[col_name].shape[0]


def _is_none(df, col_name=""):
    """find none ratio in a specific columns

    Args:
        df ([type]): [description]
        col_name (str, optional): [description]. Defaults to "".

    Returns:
        ratio: none ration in the columns
                1 means all the columns is none
                0 means non none
    """
    df_clean, _ = _is_duplicated(df)
    none_element = df_clean[col_name][df_clean[col_name].isnull()]
    ratio = len(none_element) / df_clean.shape[0]
    return ratio


def proba_model(col, mean, std, tresh=6):
    """cutting distribution between mean-3*std and mean+3*std

    Args:
        df ([type]): [description]
        col_name ([type]): [description]
        mean ([type]): [description]
        std ([type]): [description]
        tresh (int, optional): [description]. Defaults to 6.

    Returns:
        [type]: [description]
    """
    upper_bound = mean + tresh * std
    lower_bound = mean - tresh * std
    idx = col[
        ~((col > lower_bound) * (col < upper_bound))
    ].index  # trancate values from the column
      # clean dataframe
    return idx

# Possibilité d'améliorer
# Threshold for anomalie is fixed at Q_1 = round(np.percentile(unique_counts, 5)), could be improved. 
# DBSCAN for example on the number of occurences on words.

def uncorrect_grammar(df_names, cluster, min_occurence):
    """index of element

    Args:
        df_names ([type]): [description]
        cluster ([type]): [description]
        min_occurence (int): [min # of répétition of a label to be considered an error]

    Returns:
        [type]: [description]
    """
    words = np.asarray(df_names)
    unique_words, unique_counts = np.unique(df_names, return_counts=True)
    index_In_words = []
    for w in cluster:
        count = unique_counts[np.where(unique_words == w)[0]][0]
        if count <= min_occurence:
            index_In_words = index_In_words + np.ndarray.tolist(np.where(words == w)[0])
    return index_In_words


def index_uncorrect_grammar(df_State):
    df_State_unique = np.unique(df_State)
    words = np.asarray(df_State_unique) #So that indexing with a list will work
    lev_similarity = np.array([[SequenceMatcher(None, w1, w2).ratio() for w1 in words] for w2 in words])
    affprop = AffinityPropagation(affinity = "precomputed", damping=0.5)
    affprop.fit(lev_similarity)
    list_uncorrect = []
    if len(np.unique(affprop.labels_)) == 1:
        return list_uncorrect
    else:
        for cluster_id in np.unique(affprop.labels_):
            cluster = np.unique(words[np.nonzero(affprop.labels_ == cluster_id)])
            if len(cluster) > 1:
                list_uncorrect = list_uncorrect + uncorrect_grammar(df_State, cluster, 10)
    return list_uncorrect

def _duplicated_idx(df):
    df_new = df.drop(["Unnamed: 0"], axis=1)
    return df_new.duplicated()

In [ ]:
data = Data('improved_data_quality\data_avec_erreurs_wasserstein.csv')
data.set_profile()

In [ ]:
data.firstpass()

In [ ]:
mca = prince.MCA(n_components=2, n_iter=3, copy=True, check_input=True, engine='auto', random_state=42)
modified_df = data.data.drop('ProductGroup', axis=1).fillna(method='backfill', axis=1, limit=10).iloc[:2000:20]
mca = mca.fit()

In [ ]:
 ax = mca.plot_coordinates(
...     X=data.data.drop('ProductGroup', axis=1).fillna(method='backfill', axis=1, limit=10).iloc[:2000:20],
...     ax=None,
...     figsize=(12, 12),
...     show_row_points=True,
...     row_points_size=10,
...     show_row_labels=False,
...     show_column_points=True,
...     column_points_size=30,
...     show_column_labels=False,
...     legend_n_cols=2
... )

In [ ]:
clf = COPOD()
clf.fit(data.data[['Unnamed: 0', 'YearMade']])
results = clf.decision_scores_
arg = results.argsort()

In [ ]:
clf = IForest()
clf.fit(data.data[['Unnamed: 0', 'YearMade']])

In [ ]:
arg = clf.decision_scores_.argsort()

In [ ]:
data.data['YearMade'][arg]

In [ ]:
for number_table in data.data

In [ ]:
data.data['SalePrice']

In [ ]:
res = proba_model(data.data['SalePrice'][data.data['SalePrice'].notna()], data.profile[2]._mean, data.profile[2]._std, tresh=6)

In [ ]:
np.sort(data.data['SalePrice'][res])

In [ ]:
data.data['fiBaseModel']

In [ ]:
n_duped_idx = ~_duplicated_idx(data.data)
clean_df = data.data[n_duped_idx]
clean_data_na = clean_df['fiModelSeries'][clean_df['fiModelSeries'].notna().values]


idx = index_uncorrect_grammar(clean_data_na)

#idx = proba_model(clean_data_na, \
#                data.profile['YearMade']._mean, data.profile['YearMade']._std)
idx = clean_data_na.iloc[idx].index
idx

In [ ]:
data.data['fiModelSeries'].iloc[idx]

In [ ]:
data.data['fiModelSeries'][data.data['fiModelSeries'].notna().values]

In [ ]:
data.data['fiSecondaryDesc'].iloc[idx]

In [ ]:
data.data['fiSecondaryDesc'][data.data['fiSecondaryDesc'].notna().values]

In [ ]:
data.data['YearMade'][idx]

In [ ]:
idx = index_uncorrect_grammar(column_test.iloc[column_test.notna().values]) #get the non duped indexes and not na from a column
idx = column_test.iloc[~column_test.isna().values].iloc[idx].index
idx
column_test[idx]

In [ ]:
clean_data_na

In [ ]:
check_data_type(data.data['YearMade'])

In [ ]:
for idx in data.bad_index['idx']:
    print(idx)
    try:
        data.good_index.remove(idx)
    except:
        pass

In [ ]:
len(data.good_index)

In [ ]:
df_entier_nombre = data.data.iloc[data.good_index].drop('ProductGroup', axis=1).drop('saledate', axis=1)
for name_col in data.data.columns:
    if data.profile[name_col]._col_type in [type(int()), type(float())]:
        df_entier_nombre.drop(name_col, axis=1, inplace=True)

In [ ]:
le = preprocessing.LabelEncoder()


le.fit(df_entier_nombre.stack(dropna=False).reset_index(drop=True))
for col in df_entier_nombre.columns:
    df_entier_nombre[col] = le.transform(df_entier_nombre[col])

pca = PCA(n_components=10)
pca.fit(df_entier_nombre)

cov = np.abs(pca.get_covariance())
variance = np.diag(cov)
corr = (cov / np.dot(variance[:, np.newaxis], variance[np.newaxis, :]))


df_cov = pd.DataFrame({col: cov[:, idx] for idx, col in enumerate(df_entier_nombre.columns)}, index=[col for col in df_entier_nombre.columns])
df_corr = pd.DataFrame({col: corr[:, idx] for idx, col in enumerate(df_entier_nombre.columns)}, index=[col for col in df_entier_nombre.columns])

In [ ]:
plt.rcParams["figure.figsize"] = (40, 40)
plt.pcolor(df_cov, cmap='gray')
plt.yticks(np.arange(0.5, len(df_cov.index), 1), df_cov.index)
plt.xticks(np.arange(0.5, len(df_cov.columns), 1), df_cov.columns)

plt.show()

In [ ]:
from scipy.sparse import linalg, eye, csr_matrix
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import pairwise_distances

csr_matrix(1 - pairwise_distances(df_entier_nombre['state'], metric='hamming'))

In [ ]:
def _col_str_to_nb(col):
    matrix = np.array([[SequenceMatcher(None, w1, w2).ratio() for w1 in col] for w2 in col]).sum(axis=0)
    dict_pairwise = {key: value for key, value in zip(col, matrix)}
    return dict_pairwise
_col_str_to_nb(df_entier_nombre[cols[0]].stack(dropna=True).reset_index(drop=True).unique())

In [ ]:
df_entier_nombre[cols[2]].stack(dropna=True).reset_index(drop=True).unique().shape

In [ ]:
cols = []
for i, col in enumerate(df_cov.columns):
    print(col)
    cols.append(df_cov.columns[np.abs(cov[:, i]) > np.abs(cov[:, i]).mean() + 0.5*np.abs(cov[:, i]).std()].values.tolist())
    if col not in cols[i]:
        cols[i].append(col)
    print(cols[i])

In [ ]:
clf = IForest(contamination = 0.01, n_jobs = -1)

clf.fit(df_entier_nombre[cols[2]])

In [ ]:
clf = COPOD()
clf.fit(df_entier_nombre[cols[2]])

In [ ]:
proba = clf.predict_proba(df_entier_nombre[cols[2]])

In [ ]:
bd_idx2 = np.where(proba[:, 1] >= 0.95)[0]

In [ ]:
bd_idx = clf.predict(df_entier_nombre[cols[2]])

In [ ]:
data.data.iloc[data.good_index].iloc[bd_idx2][cols[2]]

In [ ]:
data.data.iloc[data.good_index][bd_idx == 0][cols[0]]

# Word embedding

In [ ]:
from gensim import corpora
from gensim import models

In [ ]:
df_document = data.data.iloc[data.good_index].drop('ProductGroup', axis=1).drop('saledate', axis=1)
for name_col in data.data.columns:
    if data.profile[name_col]._col_type in [type(int()), type(float())]:
        df_document.drop(name_col, axis=1, inplace=True)

In [ ]:
document = []
for name_col in df_document.columns:
    document.append(df_document[name_col].fillna('Nan').tolist())

In [ ]:
document

In [ ]:
dictionary = corpora.Dictionary(document)

corpus = [dictionary.doc2bow(text) for text in document]

In [ ]:

model = models.LsiModel(corpus, id2word=dictionary, num_topics=300)

In [ ]:
corpus_lsi = model[corpus]

for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

In [ ]:
from gensim.models import Word2Vec


df_document = data.data.iloc[data.good_index].drop('ProductGroup', axis=1).drop('saledate', axis=1)
for name_col in data.data.columns:
    if data.profile[name_col]._col_type in [type(int()), type(float())]:
        df_document.drop(name_col, axis=1, inplace=True)
document = []
for name_col in df_document.columns:
    document.append(df_document[name_col].fillna('Nan').tolist())
tokenized_sentences = document
model = model = Word2Vec(tokenized_sentences, vector_size=100, window=2, min_count=2, workers=6)



In [ ]:
def _string_to_nbr(df):
    """Convert a DataFrame (which may have multiple columns) of strings into a return df
    with vectors inside.
    Args:
        df ([DataFrame]): [DataFrame containing strings]
    Returns:
        [DataFrame]: [DataFrame converted into vectors]
    """
    document = []
    for col in df.columns:
        document.append(df[col].dropna().tolist())
    tokenized_sentences = document
    model = Word2Vec(tokenized_sentences, vector_size=100, window=2, min_count=0, workers=6)
    df = df.applymap(lambda x: map_model(model, x))
    import ipdb; ipdb.set_trace()
    return df


In [ ]:
def map_model(model, x):
    try:
        return model.wv[x]
    except:
        return np.nan

In [ ]:
print(model.wv['Medium'])
print(model.wv.most_similar('Medium', topn=50))

In [ ]:
df_document.isnull().sum()

In [ ]:
df2 = df_document
df_nb = _string_to_nbr(df2)

In [ ]:
df_nb

In [ ]:
glove_vectors.most_similar('excavator')

In [ ]:

cols = [[col] for col in data.data.columns.values]
tokenized_sentences_col = cols
lsi = Word2Vec(tokenized_sentences_col, vector_size=20, window=2, min_count=0, workers=6)

print(lsi.wv.most_similar('saledate'))

In [ ]:
print(lsi.wv.most_similar('YearMade'))

In [ ]:
def _tendancy_detection(df, thresh):
    """
    df = dataframe
    thresh = threshold for the tendancy detection 
    return:
            dictionnaire with the pair of columns and the anomaly index detected.
    """
    dictionnaire_anomalie_tendance = {}
    
    for w1 in df:
        for w2 in df:
            range_anomalie = 0
            proportion = np.shape(np.where(df[w1] <  df[w2])[0])[0]/len(df)
            if  proportion > thresh:
                range_anomalie = np.shape(np.where(df[w1] > df[w2])[0])[0]
                if range_anomalie > 0:
                    dictionnaire_anomalie_tendance[(w1, w2)] = np.ndarray.tolist(np.where(df[w1] > df[w2])[0])
    return dictionnaire_anomalie_tendance  

def _year(x):
    return x.year + x.month / 12 + x.day / 365


def _to_date_and_float(df):
    pd.options.mode.chained_assignment = None 
    for col in df.columns:
        if df[col].dtype == "object":
            try:
                df[col] = pd.to_datetime(df[col])
            except ValueError:
                pass
    for col in df.columns:
        if df[col].dtype == "datetime64[ns]":
            df[col] = df[col].apply(lambda x: _year(x))
        elif df[col].dtype == "int64":
            df[col] = df[col].apply(lambda x: float(x))
        else:
            pass
    return df


In [ ]:
pd.to_datetime(data.data['saledate'])

In [ ]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    # extract the words & their vectors, as numpy arrays
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  # fixed-width numpy strings

    # reduce using t-SNE
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')


def plot_with_matplotlib(x_vals, y_vals, labels):
    import matplotlib.pyplot as plt
    import random

    random.seed(0)

    plt.figure(figsize=(12, 12))
    plt.scatter(x_vals, y_vals)

    #
    # Label randomly subsampled 25 data points
    #
    indices = list(range(len(labels)))
    selected_indices = random.sample(indices, 25)
    for i in selected_indices:
        plt.annotate(labels[i], (x_vals[i], y_vals[i]))

try:
    get_ipython()
except Exception:
    plot_function = plot_with_matplotlib
else:
    plot_function = plot_with_plotly

plot_function(x_vals, y_vals, labels)

In [ ]:
df_nb.columns

In [ ]:
df_l2 = pd.DataFrame({col: l2_cov[:, idx] for idx, col in enumerate(df_nb.columns)}, index=[col for col in df_nb.columns])

In [ ]:
plt.rcParams["figure.figsize"] = (40, 40)
plt.pcolor(df_l2, cmap='gray')
plt.yticks(np.arange(0.5, len(df_l2.index), 1), df_l2.index)
plt.xticks(np.arange(0.5, len(df_l2.columns), 1), df_l2.columns)

plt.show()

In [ ]:
l2_cov = np.zeros((df_nb.columns.shape[0], df_nb.columns.shape[0]))
for i, col1 in enumerate(df_nb.columns):
    for j, col2 in enumerate(df_nb.columns):
        l2_cov[i, j] = np.linalg.norm(np.array(df_nb[col1].values.tolist()) - np.array(df_nb[col2].values.tolist()))
    print(i)

In [ ]:
pca = PCA(n_components=1)
pca.fit_transform(np.array(df_nb[col1].values.tolist()))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.array(df_nb['UsageBand'].dropna().values.tolist()).mean(axis=1), 'b.')
plt.show()

In [ ]:
df_usage_band_na = df_nb['UsageBand']
df_usage_band_na['true_index'] = df_nb.index.values.tolist()

In [ ]:
true_idx = df_usage_band_na.dropna().index

In [ ]:
true_idx

In [ ]:
pd.DataFrame(np.array(df_nb['UsageBand'].dropna().values.tolist()).mean(axis=1))

In [ ]:
col = 'UsageBand'
df_usage_band_na = df_nb[col]
df_usage_band_na['true_index'] = df_nb.index.values.tolist()
true_idx = df_usage_band_na.dropna().index
col_name = 0
dff = pd.DataFrame(np.array(df_nb[col].dropna().values.tolist()).mean(axis=1))
mean_df, std_df=dff[col_name].mean(), dff[col_name].std()
ind = _z_score(col=dff[col_name], mean=mean_df, std=std_df, tresh=3)
df2.loc[true_idx[dff[col_name].loc[ind].index.values.tolist()]][col]
#pd.DataFrame((dff[["Unnamed: 0", col_name]]).loc[ind].dropna(axis=0)).shape
#pd.DataFrame((dff[["Unnamed: 0", col_name]]).loc[ind].dropna(axis=0)).T

In [ ]:
print(len(ind))
print(mean_df, std_df)

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(dff[ col_name], 'b.', label="all")
plt.plot(dff[ col_name].loc[ind], 'r*', label="outlier")
plt.legend()
plt.show()

In [ ]:
df2.loc[true_idx[dff[ col_name].loc[ind].index.values.tolist()]]['UsageBand']

In [ ]:
np.unique(df2['fiModelDesc'].dropna())

In [ ]:
def _z_score(col, mean, std, tresh=6):
    """cutting distribution between mean-6*std and mean+6*std
    Args:
        df ([type]): [description]
        col_name ([type]): [description]
        mean ([type]): [description]
        std ([type]): [description]
        tresh (int, optional): [description]. Defaults to 6.
    Returns:
        [type]: [description]
    """
    upper_bound = mean + tresh * std
    lower_bound = mean - tresh * std
    idx = col[
        ~((col > lower_bound) & (col < upper_bound))
    ].index  # trancate values from the column
    print(mean, std)
    return idx


In [ ]:
!dir

In [ ]:
df_marque = pd.read_csv(r'improved_data_quality\data\echantillon.csv')

In [ ]:
df_marque_nb = _string_to_nbr(df_marque)

In [ ]:
df_marque_nb.columns

In [ ]:
col = 'MARQUE'
df_usage_band_na = df_marque_nb[col]
df_usage_band_na['true_index'] = df_nb.index.values.tolist()
true_idx = df_usage_band_na.dropna().index
col_name = 0
dff = pd.DataFrame(np.array(df_marque_nb[col].dropna().values.tolist()).mean(axis=1))
mean_df, std_df=dff[col_name].mean(), dff[col_name].std()
ind = _z_score(col=dff[col_name], mean=mean_df, std=std_df, tresh=6)
df_marque.loc[true_idx[dff[col_name].loc[ind].index.values.tolist()]]

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(dff[ col_name], 'b.', label="all")
plt.plot(dff[ col_name].loc[ind], 'r*', label="outlier")
plt.legend()
plt.show()